In [27]:
import pandas as pd
import os

# How many pubmed IDs in this study

In [28]:
orifile="/scratch/Shares/dowell/dbnascent/out/meta_analysis/snpconnecter/inputdata/EFO_0000565_associations_export.tsv"

In [29]:
orifiledf = pd.read_csv(orifile, sep="\t")
len(orifiledf["pubmedId"].unique())

35

In [30]:
mainindir="/scratch/Shares/dowell/dbnascent/out/meta_analysis/snpconnecter/outputdata/"
rootname="EFO_0000565"
outdir = mainindir+"tissues_"+rootname+"/"
if not os.path.exists(outdir):
    os.makedirs(outdir)


# Read in all the data that came from the GWAStogenes.sh script 

In [31]:
def snpclosestgenedf(mainindir, rootname, tissue):
    snptobedfile=mainindir+rootname+"_"+tissue+"/"+rootname+".snp_to_bed.txt"
    snptobeddf = pd.read_csv(snptobedfile, sep="\t")
    closestgenetosnp=mainindir+rootname+"_"+tissue+"/"+rootname+".closestgene.sorted.bed"
    closestgenetosnpdf = pd.read_csv(closestgenetosnp, sep="\t", names=["snp_chr", "snp_start", "snp_stop", "closest_gene_chr", "closest_gene_start", "closest_gene_stop", "closest_gene_name","score", "strand", "closest_distance"], index_col=False)
    closestgenetosnpdf[["closest_gene", "closest_transcript_id"]] = closestgenetosnpdf['closest_gene_name'].str.split(':', n=1, expand=True)
    masterdf = snptobeddf.merge(closestgenetosnpdf, on=["snp_chr", "snp_start"])
    print ("unique snps in bidirectionals", masterdf["rsID"].nunique())
    return masterdf
    

def merge_dfs_outer(mainindir, rootname, tissue):
    snptobedfile=mainindir+rootname+"_"+tissue+"/"+rootname+".snp_to_bed.txt"
    bedtobidirfile=mainindir+rootname+"_"+tissue+"/"+rootname+".snpinbidir.df"
    bidirtogenefile=mainindir+rootname+"_"+tissue+"/"+rootname+".genelist.txt.df"
    snptobeddf = pd.read_csv(snptobedfile, sep="\t")
    checkedbidirs = readsnpsandgeneschecked()
    bedtobidirdf = pd.read_csv(bedtobidirfile)
    bidirtogenedf = pd.read_csv(bidirtogenefile)
    bedtobidirdf["bidirtesed_for_links"]= bedtobidirdf["bidirname"].isin(checkedbidirs)
    masterdf = snptobeddf.merge(bedtobidirdf, on=["snp_chr", "snp_start"], how="outer")
    masterdf = masterdf.merge(bidirtogenedf, left_on="bidirname", right_on="transcript_2", how="outer")
    masterdf_mini = masterdf[["riskAllele", "rsID", 'snp_chr', 'snp_start', "bidirname", "distance_tss", "transcript_1", "gene", "bidirtesed_for_links"]].drop_duplicates()
    closestgenetosnpdf = snpclosestgenedf(mainindir, rootname, tissue)
    backtoallsnps = closestgenetosnpdf.merge(masterdf_mini, how="outer", on=["snp_chr", "snp_start", "riskAllele", "rsID"])
    backtoallsnps["snp_in_gene"] = backtoallsnps["closest_distance"]==0
    return backtoallsnps

#
def readsnpsandgeneschecked():
    #only genes and snps with expression were checked for correlation
    filename="/scratch/Shares/dowell/rutendo/projects/DBNascent_Analysis/data/counts/normalized/genes_inter_bidir_filtered.tsv.gz"
    checkedforlinksdf = pd.read_csv(filename, compression='gzip', sep="\t",low_memory=False)
    bidircheckedforlinksdf = checkedforlinksdf[checkedforlinksdf["gene_transcript"].str.startswith("chr")]filename="/scratch/Shares/dowell/rutendo/projects/DBNascent_Analysis/data/counts/normalized/genes_inter_bidir_filtered.tsv.gz"
    checkedforlinksdf = pd.read_csv(filename, compression='gzip', sep="\t",low_memory=False)
    return bidircheckedforlinksdf["gene_transcript"].unique()
    
    

In [32]:
tissue="blood"
allsnps_one_tissue = merge_dfs_outer(mainindir, rootname, tissue)
allsnps_one_tissue

unique snps in bidirectionals 7805


,Unnamed: 0,riskAllele,locations,snp_chr,snp_start,rsID,change,stop,snp_stop,closest_gene_chr,...,strand,closest_distance,closest_gene,closest_transcript_id,bidirname,distance_tss,transcript_1,gene,bidirtesed_for_links,snp_in_gene
0,0,rs7090445-?,10:61961417,chr10,61961417,rs7090445,?,61961418,61961418,chr10,...,+,88689,ARID5B,NM_001244638.2,chr10:61961221-61961505,NaN,NaN,NaN,False,False
1,63,rs7090445-?,10:61961417,chr10,61961417,rs7090445,?,61961418,61961418,chr10,...,+,88689,ARID5B,NM_001244638.2,chr10:61961221-61961505,NaN,NaN,NaN,False,False
2,1,rs2665658-A,2:60599667,chr2,60599667,rs2665658,A,60599668,60599668,chr2,...,-,71883,BCL11A,NR_175964.1,NaN,NaN,NaN,NaN,NaN,False
3,2,rs2665658-A,2:60599667,chr2,60599667,rs2665658,A,60599668,60599668,chr2,...,-,71883,BCL11A,NR_175964.1,NaN,NaN,NaN,NaN,NaN,False
4,3,rs4869742-T,6:151586613,chr6,151586613,rs4869742,T,151586614,151586614,chr6,...,+,0,CCDC170,NM_025059.4,NaN,NaN,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13082,2110,rs131821-AT,22:50511648,chr22,50511648,rs131821,AT,50511649,50511649,chr22,...,+,0,NCAPH2,NM_014551.5,chr22:50511408-50511748,NaN,NaN,NaN,False,True
13083,2111,rs11715604-?,3:136870707,chr3,136870707,rs11715604,?,136870708,136870708,chr3,...,-,9395,NCK1-DT,NR_110175.1,chr3:136869554-136871154,NaN,NaN,NaN,False,False
13084,2112,rs9421684-A,10:80702590,chr10,80702590,rs9421684,A,80702591,80702591,chr10,...,+,48857,LINC02655,NR_155749.1,NaN,NaN,NaN,NaN,NaN,False
13085,2113,rs11187157-C,10:92742487,chr10,92742487,rs11187157,C,92742488,92742488,chr10,...,+,46841,HHEX,NM_002729.5,NaN,NaN,NaN,NaN,NaN,False


In [33]:
allsnps_one_tissue.columns

Index(['Unnamed: 0', 'riskAllele', 'locations', 'snp_chr', 'snp_start', 'rsID',
       'change', 'stop', 'snp_stop', 'closest_gene_chr', 'closest_gene_start',
       'closest_gene_stop', 'closest_gene_name', 'score', 'strand',
       'closest_distance', 'closest_gene', 'closest_transcript_id',
       'bidirname', 'distance_tss', 'transcript_1', 'gene',
       'bidirtesed_for_links', 'snp_in_gene'],
      dtype='object')

# Output what we know about all snps in this GWAS catalog for one tissue

In [34]:

def informationforpub(tissue, backtoallsnps):
    print ("riskAlleles", len(backtoallsnps["riskAllele"].unique()))
    print ("snp locations", len(backtoallsnps["locations"].unique()))
    print ("rsID", len(backtoallsnps["rsID"].unique()))
    print ("closest genes", len(backtoallsnps["closest_gene"].unique()))
    ingenesnpsdf = backtoallsnps[backtoallsnps["closest_distance"]==0]
    
    print ("snps in genes", ingenesnpsdf["rsID"].nunique())
    print ("closest genes when snp is in a gene", len(ingenesnpsdf["closest_gene"].unique()))
    notingenesnpsdf = backtoallsnps[backtoallsnps["closest_distance"]!=0]
    print ("snps not in  genes", notingenesnpsdf["rsID"].nunique())
    print ("closest genes when snp is not in a gene", len(notingenesnpsdf["closest_gene"].unique()))
    inabidir = backtoallsnps[~backtoallsnps["bidirname"].isna()]
    print ("snps in any birectional", inabidir["rsID"].nunique())
    inabidirnotgene = notingenesnpsdf[~notingenesnpsdf["bidirname"].isna()]
    inabidirinagene = ingenesnpsdf[~ingenesnpsdf["bidirname"].isna()]
    inabidirinagenetestedforsnps = backtoallsnps[backtoallsnps["bidirtesed_for_links"]==True]
    inabidirinagenetestedforsnps2 = inabidirnotgene[inabidirnotgene["bidirtesed_for_links"]==True]
    inabidirinagenetestedforsnps3 = inabidirinagene[inabidirinagene["bidirtesed_for_links"]==True]
    print ("snps in any birectional that's not in a gene", inabidirnotgene["rsID"].nunique())
    print ("snps in any birectional that's in a gene", inabidirinagene["rsID"].nunique())
    #/scratch/Shares/dowell/rutendo/projects/DBNascent_Analysis/data/counts/normalized/genes_inter_bidir_filtered.tsv.gz
    print ("snps in any birectional that was tested for correlations", inabidirinagenetestedforsnps["rsID"].nunique())
    print ("snps in any birectional that was tested for correlations (snp not in gene)", inabidirinagenetestedforsnps2["rsID"].nunique())
    print ("snps in any birectional that was tested for correlations (snp in gene)", inabidirinagenetestedforsnps3["rsID"].nunique())  
    print ("closeset genes when snp in any birectional", len(inabidirnotgene["closest_gene"].unique()))
    inabidirlinkedtoagene = inabidirnotgene[~inabidirnotgene["transcript_1"].isna()]
    print ("snps in any birectional and snp linked to gene", inabidirlinkedtoagene["rsID"].nunique())
    print ("closeset genes when snp in any birectional and snp linked to gene", len(inabidirlinkedtoagene["closest_gene"].unique()))
    print ("nascentlinked genes when snp in any birectional and snp linked to gene", len(inabidirlinkedtoagene["transcript_1"].unique()))
    inabidirlinkedtoagene = inabidirnotgene[~inabidirnotgene["transcript_1"].isna()]
    repogene = [gene for gene in backtoallsnps["gene"].unique() if gene not in backtoallsnps["closest_gene"].unique()]
    closestgene = [gene for gene in backtoallsnps["closest_gene"].unique() if gene not in backtoallsnps["gene"].unique()]
    both = [gene for gene in backtoallsnps["closest_gene"].unique() if gene in backtoallsnps["gene"].unique()]
    print ("for all snps intergentic and geneic")
    print ("\tgenes that are nascentlinked and are not closeset genes", len(repogene))
    print ("\tgenes that are both in closeset genes and nascentlinked", len(both))
    print ("\tgenes that are closeset genes not nascentlinked", len(closestgene))
    repogene = [gene for gene in notingenesnpsdf["gene"].unique() if gene not in notingenesnpsdf["closest_gene"].unique()]
    closestgene = [gene for gene in notingenesnpsdf["closest_gene"].unique() if gene not in notingenesnpsdf["gene"].unique()]
    both = [gene for gene in notingenesnpsdf["closest_gene"].unique() if gene in notingenesnpsdf["gene"].unique()]
    print ("for snps that are intergentic")
    print ("\tgenes that are nascentlinked and are not closeset genes", len(repogene))
    print ("\tgenes that are both in closeset genes and nascentlinked", len(both))
    print ("\tgenes that are closeset genes not nascentlinked", len(closestgene))
    repogene = [gene for gene in inabidirnotgene["gene"].unique() if gene not in inabidirnotgene["closest_gene"].unique()]
    closestgene = [gene for gene in inabidirnotgene["closest_gene"].unique() if gene not in inabidirnotgene["gene"].unique()]
    both = [gene for gene in inabidirnotgene["closest_gene"].unique() if gene in inabidirnotgene["gene"].unique()]
    print ("for only snps in bidirs")
    print ("\tgenes that are nascentlinked and are not closeset genes", len(repogene))
    print ("\tgenes that are both in closeset genes and nascentlinked", len(both))
    print ("\tgenes that are closeset genes not nascentlinked", len(closestgene))

,Unnamed: 0,riskAllele,locations,snp_chr,snp_start,rsID,change,stop,snp_stop,closest_gene_chr,...,strand,closest_distance,closest_gene,closest_transcript_id,bidirname,distance_tss,transcript_1,gene,bidirtesed_for_links,snp_in_gene


In [35]:
backtoallsnps = allsnps_one_tissue

In [36]:
notingenesnpsdf = backtoallsnps[backtoallsnps["closest_distance"]!=0]
notingenesnpsdf["closest_gene"].unique()

array(['ARID5B', 'BCL11A', 'MIR4522', 'PCAT29', 'LRIG1', 'LOC105374297',
       'CFAP100-DT', 'ENPP5', 'ADAMTS14', 'SLC39A10', 'AMD1', 'GRAMD1B',
       'BCL2L11', 'FIGNL1', 'LINC00977', 'CDKN2A', 'FAM53B', 'ELK3',
       'CEBPE', 'TEX41', 'IKZF1', 'CPSF2', 'MYRIP', 'MMADHC', 'PDE4B',
       'PARP12', 'TIGAR', 'MIR4465', 'CFHR4', 'METTL4', 'HS6ST1', 'SP4',
       'TP63', 'OR5AL1', 'GPATCH2L', 'INTS10', 'LINC02494', 'KCNA3',
       'ADAT2', 'HTR1B', 'KLHL5', 'IRF4', 'ACOXL-AS1', 'IRF8', 'CCDC18',
       'TATDN3', 'LINC02229', 'HLA-DOB', 'SLC25A51P1', 'LHFPL3', 'TPK1',
       'LOC101929998', 'SEC23IP', 'MIR8068', 'FREM2', 'SMAD3-DT', 'HAP1',
       'HLA-DRB1', 'ODF1', 'CASC19', 'DMRTA1', 'BMF', 'MNS1', 'PMAIP1',
       'PHLPP1', 'EOMES', 'SERPINB6', 'LPP-AS1', 'CSRNP1', 'QPCT',
       'CFLAR-AS1', 'SP140L', 'SEPTIN2', 'CDKN2B-AS1', 'PRKD2', 'MDS2',
       'TMPRSS5', 'CXXC1', 'ZBTB7A', 'ODF3B', 'LINC02363', 'CAMK2D',
       'IPCEF1', 'C11orf21', 'CASP8', 'HLA-DQB1', 'MYNN', 'BAK1',
      

In [37]:
informationforpub("blood", backtoallsnps)

riskAlleles 8513
snp locations 7805
rsID 7805
closest genes 1386
snps in genes 3245
closest genes when snp is in a gene 676
snps not in  genes 4560
closest genes when snp is not in a gene 949
snps in any birectional 1061
snps in any birectional that's not in a gene 579
snps in any birectional that's in a gene 482
snps in any birectional that was tested for correlations 393
snps in any birectional that was tested for correlations (snp not in gene) 391
snps in any birectional that was tested for correlations (snp in gene) 2
closeset genes when snp in any birectional 349
snps in any birectional and snp linked to gene 126
closeset genes when snp in any birectional and snp linked to gene 99
nascentlinked genes when snp in any birectional and snp linked to gene 259
for all snps intergentic and geneic
	genes that are nascentlinked and are not closeset genes 217
	genes that are both in closeset genes and nascentlinked 43
	genes that are closeset genes not nascentlinked 1343
for snps that are i

In [50]:
def tissuesummary(tissue):
    allsnps_one_tissue = merge_dfs_outer(mainindir, rootname, tissue)
    backtoallsnps = allsnps_one_tissue
    notingenesnpsdf = allsnps_one_tissue[allsnps_one_tissue["closest_distance"]!=0]
    closest_gene_tointergenicsnp = notingenesnpsdf[["closest_gene"]].drop_duplicates()
    closest_gene_tointergenicsnp.to_csv(mainindir+rootname+"_"+tissue+"/"+rootname+"closest.genelist.intergenicsnps.txt", header=False, index=False)
    notingenesnpsdf_bidirtestedforlinks = notingenesnpsdf[notingenesnpsdf["bidirtesed_for_links"]==True]
    closest_gene_tointergenicsnp_bidirtestedforlinks = notingenesnpsdf_bidirtestedforlinks[["closest_gene"]].drop_duplicates()
    closest_gene_tointergenicsnp_bidirtestedforlinks.to_csv(mainindir+rootname+"_"+tissue+"/"+rootname+"closest.genelist.intergenicsnps.bidirwasincortest.txt", header=False, index=False)
    backtoallsnps = merge_dfs_outer(mainindir, rootname, tissue)
    ingenesnpsdf = backtoallsnps[backtoallsnps["closest_distance"]==0]
    notingenesnpsdf = backtoallsnps[backtoallsnps["closest_distance"]!=0]
    inabidirnotgene = backtoallsnps[~backtoallsnps["bidirname"].isna()]
    inabidirlinkedtoagene = inabidirnotgene[~inabidirnotgene["transcript_1"].isna()]
    nsnpslinkedtogenes = inabidirlinkedtoagene["rsID"].nunique()
    nbidirslinkedtogenes = inabidirlinkedtoagene["bidirname"].nunique()
    nnascentlinkedgenes = len(inabidirlinkedtoagene["transcript_1"].unique())
    print(tissue)
    print(backtoallsnps[backtoallsnps["gene"]=="NES"])
    return nsnpslinkedtogenes, nbidirslinkedtogenes, nnascentlinkedgenes

# compare links in each tissue

In [51]:
tissues = ['blood', 'breast', 'embryo', 'heart', 'intestine', 'kidney','lung', 'prostate', 'skin', 'uterus', 'umbilical_cord']

In [52]:
nlinkedsnps = []
nlinkedbidirs = []

nlinkedgenes = []

for tissue in tissues:
    print (tissue)
    nsnpslinkedtogenes, nbidirslinkedtogenes, nnascentlinkedgenes = tissuesummary(tissue)
    nlinkedsnps.append(nsnpslinkedtogenes)
    nlinkedbidirs.append(nbidirslinkedtogenes)
    nlinkedgenes.append(nnascentlinkedgenes)
    

blood
unique snps in bidirectionals 7805
unique snps in bidirectionals 7805
blood
Empty DataFrame
Columns: [Unnamed: 0, riskAllele, locations, snp_chr, snp_start, rsID, change, stop, snp_stop, closest_gene_chr, closest_gene_start, closest_gene_stop, closest_gene_name, score, strand, closest_distance, closest_gene, closest_transcript_id, bidirname, distance_tss, transcript_1, gene, bidirtesed_for_links, snp_in_gene]
Index: []

[0 rows x 24 columns]
breast
unique snps in bidirectionals 7805
unique snps in bidirectionals 7805
breast
Empty DataFrame
Columns: [Unnamed: 0, riskAllele, locations, snp_chr, snp_start, rsID, change, stop, snp_stop, closest_gene_chr, closest_gene_start, closest_gene_stop, closest_gene_name, score, strand, closest_distance, closest_gene, closest_transcript_id, bidirname, distance_tss, transcript_1, gene, bidirtesed_for_links, snp_in_gene]
Index: []

[0 rows x 24 columns]
embryo
unique snps in bidirectionals 7805
unique snps in bidirectionals 7805
unique snps in bi

In [41]:
nlinkedgenes

[259, 144, 69, 101, 243, 189, 106, 193, 76, 233, 32]

In [42]:
nlinkedbidirs

[111, 65, 21, 36, 54, 62, 26, 44, 20, 70, 14]

In [43]:
nlinkedsnps

[126, 75, 25, 43, 63, 74, 30, 51, 30, 84, 18]

In [44]:
todf = zip(tissues, nlinkedsnps, nlinkedbidirs, nlinkedgenes)
labels = ["tissue", "snps linked","bidir linked",  "genes linked"]
tissuesnpdflinkdf = pd.DataFrame.from_records(todf, columns = labels) 

In [45]:
tissuesnpdflinkdf.to_csv(outdir+rootname+"_linkedpertissue.csv")

In [46]:
tissuesnpdflinkdf["gene per bidir"]= tissuesnpdflinkdf["genes linked"]/tissuesnpdflinkdf["bidir linked"]

In [47]:
tissuesnpdflinkdf

,tissue,snps linked,bidir linked,genes linked,gene per bidir
0,blood,126,111,259,2.333333
1,breast,75,65,144,2.215385
2,embryo,25,21,69,3.285714
3,heart,43,36,101,2.805556
4,intestine,63,54,243,4.500000
5,kidney,74,62,189,3.048387
6,lung,30,26,106,4.076923
7,prostate,51,44,193,4.386364
8,skin,30,20,76,3.800000
9,uterus,84,70,233,3.328571
